In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
pip install easybert

     |████████████████████████████████| 81kB 2.7MB/s 
     |████████████████████████████████| 71kB 3.8MB/s 
     |████████████████████████████████| 345.2MB 46kB/s 
     |████████████████████████████████| 3.2MB 33.2MB/s 
     |████████████████████████████████| 368kB 27.4MB/s 
  Created wheel for easybert: filename=easybert-1.0.3-cp36-none-any.whl size=7621 sha256=9700f4a553ec12c0aaf5021505bbbf1c2ba1add3d2292123ce3bc7b1bc376e0c
  Stored in directory: /root/.cache/pip/wheels/e4/00/9f/587ad878f58cb7255674e9027c546893e8a921b13cec954510
Successfully built easybert
ERROR: tensorflow 1.15.0 has requirement tensorboard<1.16.0,>=1.15.0, but you'll have tensorboard 1.13.1 which is incompatible.
ERROR: tensorflow 1.15.0 has requirement tensorflow-estimator==1.15.1, but you'll have tensorflow-estimator 1.13.0 which is incompatible.
  Found existing installation: tensorflow-hub 0.7.0
    Uninstalling tensorflow-hub-0.7.0:
      Successfully uninstalled tensorflow-hub-0.7.0
  Found existing installat

In [25]:
pip install nltk

In [0]:
import pandas as pd
import numpy as np
import random
import nltk
import re
import sklearn
from sklearn.metrics import precision_score,recall_score,f1_score,accuracy_score
from sklearn.ensemble import RandomForestClassifier

In [4]:
from easybert import Bert
bert = Bert("https://tfhub.dev/google/bert_multi_cased_L-12_H-768_A-12/1")

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

Instructions for updating:
Colocations handled automatically by placer.


Instructions for updating:
Colocations handled automatically by placer.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [0]:
#Importing datasets#
#Train set
file1 = open("/content/drive/My Drive/Emoji_prediction/us_train.text", "r+", encoding="utf-8")
train_text = file1.readlines()
file2 = open("/content/drive/My Drive/Emoji_prediction/us_train.labels", "r+", encoding="utf-8")
train_labels = file2.readlines()
#Development set
file3 = open("/content/drive/My Drive/Emoji_prediction/us_dev.text", "r+", encoding="utf-8")
dev_text = file3.readlines()
file4 = open("/content/drive/My Drive/Emoji_prediction/us_dev.labels", "r+", encoding="utf-8")
dev_labels = file4.readlines()
#Test set
file5 = open("/content/drive/My Drive/Emoji_prediction/us_test.text", "r+", encoding="utf-8")
test_text = file5.readlines()
file6 = open("/content/drive/My Drive/Emoji_prediction/us_test.labels", "r+", encoding="utf-8")
test_labels = file6.readlines()



train_set = []
dev_set = []
test_set = []

**Text Preprocessing**

In [35]:
#Initalizing lemmatizer and creating a list of stopwords
lemmatizer = nltk.stem.WordNetLemmatizer()
nltk.download('stopwords')
nltk.download('wordnet')
stopwords = set(nltk.corpus.stopwords.words("english"))

#Cleans reviews, removes stopwords and lemmatize to put the reviews in a form that machine can perform better
def clean_reviews(string):
    lemmatized_string = ""
    #Remove HTML information from the string
    cleaner = re.compile('<.*?>')
    processed_string = re.sub(cleaner," ", string) 
    
    #Remove URLS from the string
    processed_string = re.compile(r"https?://[A-Za-z0-9./]+").sub(" ", processed_string)
    
    #Remove digits and punctuations
    processed_string = re.compile(r"[^a-zA-Z ]").sub(" ", processed_string)
    
    #Lowercase all the words
    processed_string = processed_string.lower()
    
    #Does not append stopwords but appends other strings while lemmatizing them
    for word in processed_string.split():
        if word in stopwords:
            continue
        else:
            lemmatized_string += lemmatizer.lemmatize(word) + " "
    return lemmatized_string


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [0]:
#Train, Dev, Test sets of after data preprocessing#
train_processed_text = []
train_processed_labels = []
dev_processed_text = []
dev_processed_labels = []
test_processed_text = []
test_processed_labels = []


for i in train_text:
  train_processed_text.append(clean_reviews(i))
for i in dev_text:
  dev_processed_text.append(clean_reviews(i))
for i in test_text:
  test_processed_text.append(clean_reviews(i))
for i in train_labels:
   train_processed_labels.append(i.replace("\n", ""))
for i in dev_labels:
   dev_processed_labels.append(i.replace("\n", ""))
for i in test_labels:
   test_processed_labels.append(i.replace("\n", ""))

In [55]:
print(train_processed_text[:10])
print(train_processed_labels[0:10])

['12', '19', '0', '0', '2', '11', '0', '19', '0', '7']


**Feature** **extraction**

In [0]:
train_vector = []
test_vector = []
for i in range(5000):
  train_vector.append(bert.embed(train_processed_text[i]))
for i in range(5000):
  test_vector.append(bert.embed(test_processed_text[i]))



In [0]:
print(train_vector)
print(train_processed_labels[0:10])

**Model Implementation**

In [63]:
#Train random forest model#
rfc = RandomForestClassifier()
rfc.fit(train_vector, train_processed_labels[0:10])
display(rfc.score(train_vector, train_processed_labels[0:10]))

#Predict the test_set
y_predict = rfc.predict(test_vector)

1.0

In [68]:
#Calculate precision, recall, f1 score and accuracy of the model
precision = precision_score(test_processed_labels, y_predict, average='macro')
recall = recall_score(test_processed_labels, y_predict, average='macro')
f1 = f1_score(test_processed_labels, y_predict, average='macro')
accuracy = accuracy_score(test_processed_labels, y_predict)

#Print the evaluation measures
print ("Precision: "+str(round(precision,5)))
print ("Recall: "+str(round(recall,5)))
print ("F1-Score: "+str(round(f1,5)))
print ("Accuracy: "+str(round(accuracy,5)))

['0' '19' '0' '0' '0' '0' '0' '19' '11' '0']
['2', '10', '6', '1', '16', '17', '4', '10', '12', '18']
